In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import numpy as np
from numpy.linalg import lstsq as ldiv

def import_data(filename):
    df_ = pd.DataFrame()
    d = np.loadtxt(filename)
    df = pd.DataFrame(d, columns=['YYYYMMDD', 'HH', 'Zonal price', 'System load', 'Zonal load', 'Day of the week'])
    df_['price'] = df['Zonal price']
    return df_

filename = '/Users/sebastiansuwada/Desktop/Predictive Analytics/Naive_probabilistic_forecasts_List_3/GEFCOM.txt'
df = import_data(filename)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25968 entries, 0 to 25967
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   25968 non-null  float64
dtypes: float64(1)
memory usage: 203.0 KB


Obtain Naive forecast for prepare of Point Forcast

In [53]:
def quantreg(x, y, tau):
    def rho(r):
        temp = y - np.dot(x,r)
        return np.sum(np.abs(temp - (temp < 0) * temp / tau))
    pmean = ldiv(x, y, rcond=None)[0]
    # return minimize(rho, pmean, method='SLSQP', options={'disp': False}).x
    # return minimize(rho, pmean, method='SLSQP', tol=None, options={'maxiter': 25, 'disp': False}).x
    # return minimize(rho, pmean, method='COBYLA', options={'disp': False}).x
    # return minimize(rho, pmean, method='COBYLA', tol=None, options={'maxiter': 25, 'disp': False}).x
    # return minimize(rho, pmean, method='TNC', options={'disp': False}).x
    # return minimize(rho, pmean, method='TNC', tol=None, options={'maxiter': 25, 'disp': False}).x
    return minimize(rho, pmean, method='TNC', tol=None, options={'maxiter': 10 * len(pmean), 'disp': False}).x
    # return minimize(rho, pmean, method='L-BFGS-B', options={'disp': False}).x
    # return minimize(rho, pmean, method='L-BFGS-B', tol=None, options={'maxiter': 25, 'disp': False}).x
    # return minimize(rho, pmean, method='BFGS', options={'disp': False}).x
    # return minimize(rho, pmean, method='BFGS', tol=None, options={'maxiter': 25, 'disp': False}).x
    # return minimize(rho, pmean, method='CG', options={'disp': False}).x
    # return minimize(rho, pmean, method='CG', tol=None, options={'maxiter': 25, 'disp': False}).x
    # return minimize(rho, pmean, method='Powell', options={'disp': False}).x
    # return minimize(rho, pmean, method='Powell', tol=None, options={'maxiter': 25, 'disp': False}).x
    # return minimize(rho, pmean, method='Nelder-Mead', tol=None, options={'maxiter': 25, 'disp': False}).x
    # return minimize(rho, pmean, method='Nelder-Mead', options={'disp': False}).x

def CombinationNaiveForecast(df_1, shift1, shift2):

    # P_w = (P_(w-1)+P_(w-3))/2
    naiv1 = pd.DataFrame()
    naiv1= df_1['price'].shift(shift1)
    naiv2 = pd.DataFrame()
    naiv2= df_1['price'].shift(shift2)
    
    naiv1 = naiv1.dropna()
    naiv2 = naiv2.dropna()
    naiv1.reset_index(drop=True, inplace=True)
    naiv2.reset_index(drop=True, inplace=True)

    min_size = min(len(naiv1), len(naiv2), len(df_1))
    naiv1 = naiv1.tail(min_size)
    naiv2 = naiv2.tail(min_size)
    df_sized = df_1.tail(min_size).copy()

    dfs = [naiv1, naiv2, df_sized]
    for df in dfs:
        df.reset_index(drop=True, inplace=True)

    naivSum = pd.DataFrame()
    naivSum['price'] = (naiv1 + naiv2)/2

    return naivSum, df_sized

shift1 = 2
shift2 = 1
df_sized, naivSum = CombinationNaiveForecast(df, shift1, shift2)
print(naivSum)


        price
0       34.64
1       33.76
2       33.08
3       32.37
4       34.66
...       ...
25961  113.92
25962  107.26
25963   89.02
25964   85.40
25965   86.13

[25966 rows x 1 columns]


Now obtained point forecast can be used for Quantile Regression the dataFrame called naivSum now is our new data that we are going to work on

In [59]:
def RegressionModel(df, T, pdReal):
    quants = [0.25 ,0.75]
    hits = []
    real = pdReal['price'][T+T:].tolist()
    Xf = df['price'][T+T:].to_numpy()

    quant_forecast_upper = []
    quant_forecast_lower = []
    quant_forecast_temp = []

    for q in quants:

        # Prepare data for estimation of betas for each of "q" - quantile defined in quants table
        Y = pdReal['price'][T:T+T]
        X1 = df['price'][T:T+T]
        X0 = np.ones(np.size(X1)) # Intersept
        X = np.stack([X0,X1],axis = 1)

        # Estimate the beta for each of quantile! We are iterating over each quantile -> "q" 
        # Using defined befor quantreg function that allows to estimate betas based on quantile
        betas = quantreg(X, Y, q)

        print(betas)

        for val in range(2*T,len(df['price'])):
            quant_forecast_temp.append(betas[0]+betas[1]*df['price'][val])


    quant_forecast_upper = quant_forecast_temp[len(quant_forecast_temp)//2:]
    quant_forecast_lower = quant_forecast_temp[:len(quant_forecast_temp)//2]

    for day in range(0, len(quant_forecast_upper)):
        if real[day] < quant_forecast_upper[day] and real[day] > quant_forecast_lower[day]:
            hits.append(1)
        else:
            hits.append(0)

    print(hits.count(1)/len(hits))
    # print(hits)
    # print(quant_forecast_upper)
    # print(quant_forecast_lower)


T = 364
RegressionModel(naivSum, T, df)


/var/folders/9y/v04s15s52qd3fng308vzntx40000gn/T/ipykernel_95078/3542992026.py:12: DeprecationWarning: 'maxiter' has been deprecated in favor of 'maxfun' and will be removed in SciPy 1.11.0.
  return minimize(rho, pmean, method='TNC', tol=None, options={'maxiter': 10 * len(pmean), 'disp': False}).x


[10.7861988   0.73950868]
[10.81537329  1.01252754]
0.4256676440288454
